In [2]:
# Import needed libriary
import pandas as pd
import numpy as np

# Connect to the file
file_to_load = "./Resources/purchase_data.csv"

# Create data frame from the file 
purchase_df = pd.read_csv(file_to_load)
purchase_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [3]:
# Calculate total number of players
players_numbers = len(purchase_df['SN'].unique())
players_numbers

576

In [4]:
# identify incomplete rows 
purchase_df.count()

Purchase ID    780
SN             780
Age            780
Gender         780
Item ID        780
Item Name      780
Price          780
dtype: int64

In [5]:
# calculate number of items
unique_items = len(purchase_df['Item ID'].unique())
unique_items


179

In [6]:
# Average Purchase Price
average_price = purchase_df['Price'].mean()
average_price

3.050987179487176

In [7]:
# Total number of purchases
number_purchases = len(purchase_df['Item ID'])
number_purchases


780

In [8]:
# Total Revenue
total_revenue = purchase_df['Price'].sum()
total_revenue

2379.77

In [9]:
# build new data frame with purchasing analysis
purchase_analysis_df = pd.DataFrame({
    'Number of Unique Items': [unique_items], 
    'Average Price': [average_price],
    'Number of Purchases': [number_purchases], 
    'Total Revenue': [total_revenue]
    })
purchase_analysis_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,3.050987,780,2379.77


In [10]:
genders = purchase_df['Gender'].unique().tolist()
genders


['Male', 'Other / Non-Disclosed', 'Female']

In [11]:
# Calculating number of players by gender

male_count = len(purchase_df.loc[purchase_df['Gender'] == 'Male', :]['SN'].unique())

female_count = len(purchase_df.loc[purchase_df['Gender'] == 'Female', :]['SN'].unique())

othergender_count = len(purchase_df.loc[purchase_df['Gender'] == 'Other / Non-Disclosed', :]['SN'].unique())

total_gendercount = male_count + female_count + othergender_count


In [12]:
# creating new data frame with gender statistics
gender_analysis = pd.DataFrame({
    'Total Count': [male_count, female_count, othergender_count], 
    'Percentage of Players': [male_count/total_gendercount, female_count/total_gendercount, othergender_count/total_gendercount],
    'Gender': ['Male', 'Female', 'Other / Non-Disclosed']
}).set_index('Gender')
gender_analysis['Percentage of Players'] = gender_analysis['Percentage of Players'].map("{:.2%}".format)
gender_analysis

,Total Count,Percentage of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

In [13]:
# Purchasing Analysis by Gender
purchase_grouped_by_gender = purchase_df.groupby('Gender')

purchase_count_by_gender = purchase_grouped_by_gender[['Purchase ID']].count()

purchase_price_by_gender = purchase_grouped_by_gender[['Price']].mean()
purchase_price_by_gender['Price']= purchase_price_by_gender['Price'].map('${:.2f}'.format)

purchase_totalvalue_by_gender = purchase_grouped_by_gender[['Price']].sum().rename(columns={'Price': 'Total Purchase Value'})

purchase_totalvalue_by_gender['Avg Total Purchase per Person'] = purchase_totalvalue_by_gender['Total Purchase Value']/gender_analysis['Total Count']

purchase_totalvalue_by_gender['Total Purchase Value'] = purchase_totalvalue_by_gender['Total Purchase Value'].map('${:,.2f}'.format)

purchase_totalvalue_by_gender['Avg Total Purchase per Person'] = purchase_totalvalue_by_gender['Avg Total Purchase per Person'].map('${:,.2f}'.format)

gender_purchase_summary = pd.merge(purchase_count_by_gender, purchase_price_by_gender, on='Gender', how='left')
gender_purchase_summary = pd.merge(gender_purchase_summary, purchase_totalvalue_by_gender, on='Gender', how='left')
gender_purchase_summary

,Purchase ID,Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Purchasing Analysis (Age)

In [14]:
#Age Demographics

# Denographics analysis while removing duplicate players
unique_players_df = purchase_df.drop_duplicates(subset=['SN'])

bins = [0, 9, 14, 19, 24, 29, 34, 39, 60]
age_groups = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '>40']
unique_players_df['Age Groups'] = pd.cut(unique_players_df['Age'], bins, labels= age_groups, include_lowest=True)

purchase_df_uniqueplayers_groupedbyage = unique_players_df.groupby('Age Groups')

purchase_count_groupedbyage = purchase_df_uniqueplayers_groupedbyage[['Purchase ID']].count().rename(columns={'Purchase ID': 'Total Count(Unique Players)'})

percentage_players_groupedbyage = ((purchase_count_groupedbyage[['Total Count(Unique Players)']]/players_numbers).rename(columns={'Total Count(Unique Players)': 'Percentage of Players'}))['Percentage of Players'].map('{:.2%}'.format)

summary_by_agegroups_uniqueplayers = pd.merge(purchase_count_groupedbyage, percentage_players_groupedbyage, on='Age Groups', how='left')

summary_by_agegroups_uniqueplayers 

# Denographics analysis with all players included 

purchase_df['Age Groups'] = pd.cut(purchase_df['Age'], bins, labels= age_groups, include_lowest=True)

purchase_df_allplayers_groupedbyage = purchase_df.groupby('Age Groups')

ave_price_by_agegroupe = ((purchase_df_allplayers_groupedbyage[['Price']].mean()).rename(columns={'Price': 'Average Price'}))['Average Price'].map('${:.2f}'.format)

purchase_count_allplayers_groupedbyage = purchase_df_allplayers_groupedbyage[['Purchase ID']].count().rename(columns={'Purchase ID': 'Total Count (All Players)'})

purchase_sum_allplayers_groupedbyage = (purchase_df_allplayers_groupedbyage[['Price']].sum().rename(columns={'Price': 'Total Purchase Value'}))
summary_by_agegroups_allplayers = pd.merge(purchase_count_allplayers_groupedbyage, purchase_sum_allplayers_groupedbyage, on='Age Groups', how='left')

summary_by_agegroups_allplayers = pd.merge(summary_by_agegroups_allplayers, ave_price_by_agegroupe, on='Age Groups', how='left')

summary_by_agegroups_allplayers['Average Purchase Total per Person'] = summary_by_agegroups_allplayers['Total Purchase Value']/purchase_count_groupedbyage['Total Count(Unique Players)']

summary_by_agegroups_allplayers['Total Purchase Value'] = summary_by_agegroups_allplayers['Total Purchase Value'].map('${:,.2f}'.format)

summary_by_agegroups_allplayers['Average Purchase Total per Person'] = summary_by_agegroups_allplayers['Average Purchase Total per Person'].map('${:.2f}'.format)

summary_by_agegroups_allplayers



,Total Count (All Players),Total Purchase Value,Average Price,Average Purchase Total per Person
Age Groups,,,,
<10,23,$77.13,$3.35,$4.54
10-14,28,$82.78,$2.96,$3.76
15-19,136,$412.89,$3.04,$3.86
20-24,365,"$1,114.06",$3.05,$4.32
25-29,101,$293.00,$2.90,$3.81
30-34,73,$214.00,$2.93,$4.12
35-39,41,$147.67,$3.60,$4.76
>40,13,$38.24,$2.94,$3.19


## Top Spenders

In [38]:
by_player_df = purchase_df.groupby(['SN'])
total_purchase_value_byplayer = by_player_df[['Price']].sum().rename(columns={'Price': 'Total Purchase Value'})
summary_by_player = total_purchase_value_byplayer
summary_by_player['Purchase Count'] = by_player_df[['Purchase ID']].count()

summary_by_player['Average Purchase Price'] = by_player_df[['Price']].mean()

summary_by_player = summary_by_player.sort_values(by=['Total Purchase Value'],ascending=False)

summary_by_player['Total Purchase Value'] = summary_by_player['Total Purchase Value'].map('${:.2f}'.format)
summary_by_player['Average Purchase Price'] = summary_by_player['Average Purchase Price'].map('${:.2f}'.format)

summary_by_player.head(5)


,Total Purchase Value,Purchase Count,Average Purchase Price
SN,,,
Lisosia93,$18.96,5,$3.79
Idastidru52,$15.45,4,$3.86
Chamjask73,$13.83,3,$4.61
Iral74,$13.62,4,$3.40
Iskadarya95,$13.10,3,$4.37


## Most Popular Items

In [57]:
by_items_df = purchase_df.groupby(['Item ID', 'Item Name'])

purchase_count_byitem = by_items_df[['Purchase ID']].count().rename(columns={'Purchase ID': 'Purchase Count'})
by_item_summary = purchase_count_byitem
by_item_summary['Item Price'] = by_items_df['Price'].mean()
by_item_summary['Total Puirchase Value'] = by_items_df['Price'].sum()
by_item_summary = by_item_summary.sort_values(by=['Purchase Count'], ascending=False)
by_item_summary['Total Puirchase Value'] = by_item_summary['Total Puirchase Value'].map('${:.2f}'.format)
by_item_summary['Item Price'] = by_item_summary['Item Price'].map('${:.2f}'.format)

by_item_summary.head(5)


,,Purchase Count,Item Price,Total Puirchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
